<a href="https://colab.research.google.com/github/RKiddle/Python-Play/blob/main/Tiny_Thai_Lessons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai gTTS gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.1 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from gtts import gTTS
import gradio as gr
import os
from google.colab import userdata

# --- SETUP ---
# Get your API key from Colab secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Use a lightweight, fast model
model = genai.GenerativeModel('gemini-pro-latest') # Changed model to gemini-pro-latest

def generate_lesson(scenario):
    """Generates a Thai lesson based on a specific scenario."""

    # 1. The Prompt: We tell Gemini strictly how to format the lesson
    prompt = f"""
    Act as a friendly Thai language tutor. I want a 'Tiny Lesson' about: "{scenario}".

    Please provide the output in this exact structure:
    1. **Core Vocabulary**: 5 essential Thai words for this scenario (Thai Script - Romanization - English Meaning).
    2. **Key Phrase**: 1 useful sentence for this situation.
    3. **Grammar Tip**: A very brief, one-sentence tip related to the phrase.
    4. **Cultural Note**: A tiny fun fact about Thai culture related to this.

    Keep it bite-sized and encouraging.
    """

    try:
        response = model.generate_content(prompt)
        lesson_text = response.text

        # 2. The Audio: We extract just the Thai parts for audio (simplified for this demo)
        # For this simple version, we will just speak the Scenario and the Key Phrase to save time
        # In a full app, you'd parse the text to speak only the Thai words.
        tts_text = f"Lesson for: {scenario}. Let's practice."

        # Generate the audio file
        tts = gTTS(lesson_text, lang='th') # Note: gTTS will try to read English text with a Thai accent if mixed.
        tts.save("lesson.mp3")

        return lesson_text, "lesson.mp3"

    except Exception as e:
        return f"Error: {str(e)}", None

# --- THE INTERFACE ---
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🇹🇭 Little Thai Lesson Generator")
    gr.Markdown("Type a scenario below (e.g., *'Ordering spicy soup'* or *'Taking a motorcycle taxi'*).")

    with gr.Row():
        inp = gr.Textbox(label="What do you want to do today?", placeholder="e.g., Buying fruit at the market")
        btn = gr.Button("Generate Lesson", variant="primary")

    out_text = gr.Markdown(label="Your Lesson")
    out_audio = gr.Audio(label="Listen")

    btn.click(fn=generate_lesson, inputs=inp, outputs=[out_text, out_audio])

# Launch the app
app.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://80c19322191f339af8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
# Verify the API key is loaded
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    print(f"API Key loaded (first 5 chars): {GOOGLE_API_KEY[:5]}...")
    genai.configure(api_key=GOOGLE_API_KEY)
    try:
        # Attempt to list models to verify API key functionality
        models = genai.list_models()
        print("API Key seems valid. Successfully listed models.")
        for m in models:
            if 'generateContent' in m.supported_generation_methods:
                print(f"  - {m.name}")
    except Exception as e:
        print(f"Error verifying API key: {e}")
else:
    print("API Key not found in Colab secrets. Please ensure it's named 'GOOGLE_API_KEY'.")

API Key loaded (first 5 chars): AIzaS...
API Key seems valid. Successfully listed models.
  - models/gemini-2.5-pro-preview-03-25
  - models/gemini-2.5-flash
  - models/gemini-2.5-pro-preview-05-06
  - models/gemini-2.5-pro-preview-06-05
  - models/gemini-2.5-pro
  - models/gemini-2.0-flash-exp
  - models/gemini-2.0-flash
  - models/gemini-2.0-flash-001
  - models/gemini-2.0-flash-exp-image-generation
  - models/gemini-2.0-flash-lite-001
  - models/gemini-2.0-flash-lite
  - models/gemini-2.0-flash-lite-preview-02-05
  - models/gemini-2.0-flash-lite-preview
  - models/gemini-2.0-pro-exp
  - models/gemini-2.0-pro-exp-02-05
  - models/gemini-exp-1206
  - models/gemini-2.0-flash-thinking-exp-01-21
  - models/gemini-2.0-flash-thinking-exp
  - models/gemini-2.0-flash-thinking-exp-1219
  - models/gemini-2.5-flash-preview-tts
  - models/gemini-2.5-pro-preview-tts
  - models/learnlm-2.0-flash-experimental
  - models/gemma-3-1b-it
  - models/gemma-3-4b-it
  - models/gemma-3-12b-it
  - models/gem

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)